In [8]:
import torch
import torch.nn as nn

# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=50, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [9]:
# Synthesize dummy data

import numpy as np

# Generate synthetic data
X_train = np.random.random((10000, 1, 48))
y_train = np.random.random((10000, 4))
y_train = y_train.reshape(-1, 1, 4)


In [14]:
# Training the Model

data = torch.tensor(X_train).float().view(-1)

# Create the model (assuming you've already defined the LSTM class)
model = LSTM()

# Define loss function and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 200
for epoch in range(epochs):
    model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                         torch.zeros(1, 1, model.hidden_layer_size))

    y_pred = model(data)

    single_loss = loss_function(y_pred, data)
    single_loss.backward()
    optimizer.step()

    # Print a simple progress bar
    progress = (epoch + 1) / epochs
    num_asterisks = int(100 * progress)
    print(f"[{'=' * num_asterisks}{'-' * (100 - num_asterisks)}] {progress * 100:.2f}%", end="\r")



In [27]:
torch.save(model)

TypeError: save() missing 1 required positional argument: 'f'

In [15]:
# Testing the Model

X_test = np.random.random((1000, 1, 48))
y_test = np.random.random((1000, 4))
y_test = y_test.reshape(-1, 1, 4)

# Assuming you've already defined the model
y_pred = model(torch.tensor(X_test).float().view(-1))


In [26]:
# Creating a Confusion Matrix

from sklearn.metrics import confusion_matrix

# Assuming y_test and y_pred are the true labels and predicted labels
y_pred_classes = np.argmax(y_pred)
y_test_classes = np.argmax(y_test)

# Create the confusion matrix
confusion_matrix_result = confusion_matrix(y_test_classes, y_pred_classes)
print("Confusion Matrix:")
print(confusion_matrix_result)



RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.